## File paths and train,validation split

In [1]:
import pandas as pd
import tqdm
import numpy as np
import pyspark
import dxpy
import dxdata
import pandas as pd
import numpy as np

labels_file = 'labels.csv'
ukb_field_to_icd10_map_file = 'icd10_codes_mod.tsv'
ubk_basket_tab_file = 'ukb_basket.tab' # file path to ukb basket download .tab format
train_proportion = 0.8 # proportion of full data set to use for training (the rest will be used for validation)
output_prefix = 'ukb_real'


## Read icd10 mapping file and defined index label link

In [2]:
icdict ={}
icdcodes = []
with open(ukb_field_to_icd10_map_file,'r') as f:
    for l in f:
        lvals=l.strip().split()
        icdict["p" + lvals[0].split(".")[1]]=lvals[5]
        icdcodes.append(lvals[5])

i = -1
label_dict = {}
with open(labels_file,'r') as f:
    for l in f:
        label_dict[l.strip().split(' ')[0]]=i
        i += 1

# hard coded sex and dob
icdict['p31'] = "sex"
icdict['p34'] = "YEAR"
icdict['p52'] = "MONTH"
icdict['p40000_i0'] = "Death"

# cancer fields
for j in range(17):
    icdict['p40005_i'+str(j)] = "cancer_date_"+str(j)
    icdict['p40006_i'+str(j)] = "cancer_type_"+str(j)

# cancer hes fields 
#for j in range(213):
#    icdict['f.41270.0.'+str(j)] = "hicd_"+str(j)
#    icdict['f.41280.0.'+str(j)] = "hicd_date_"+str(j)

icdict['p53_i0'] = "assessment_date"
icdict['p21001_i0']="BMI"
icdict['p1239_i0']="smoking"
icdict['p1558_i0']="alcohol"

len_icd = len(icdcodes)
#icdcodes.extend(['Death','assessment_date']+['cancer_date_'+str(j) for j in range(17)]+['hicd_date_'+str(j) for j in range(213)])
icdcodes.extend(['Death','assessment_date']+['cancer_date_'+str(j) for j in range(17)])


## Retrieve ukb fields from SQL database

In [3]:
def get_database_id():
    dispensed_dataset = dxpy.find_one_data_object(
        typename='Dataset', 
        name='app*.dataset', 
        folder='/', 
        name_mode='glob')
    dispensed_dataset_id = dispensed_dataset['id']
    return(dispensed_dataset_id)

sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

dispensed_dataset_id = get_database_id()

dataset = dxdata.load_dataset(id=dispensed_dataset_id)
participant = dataset['participant']

In [4]:
fields_to_get = ["eid"] + list(icdict.keys())

In [5]:
dd_sql = participant.retrieve_fields(names=fields_to_get, 
                                 coding_values = "raw",
                                 engine=dxdata.connect(dialect="hive+pyspark"))
dd = dd_sql.toPandas()

In [6]:
dd = dd.set_index("eid")

## format for delphi

In [7]:
data_list = []

dd = dd.rename(columns=icdict)
dd.dropna(subset=['sex'], inplace=True)
dd['sex'] += 1
dd = dd[[col for col in dd.columns if not col.startswith('f.')]]
dd['dob'] =  pd.to_datetime(dd[['YEAR', 'MONTH']].assign(DAY=1))
dd[icdcodes] = dd[icdcodes].apply(pd.to_datetime, format="%Y-%m-%d")
dd[icdcodes]=dd[icdcodes].sub(dd['dob'], axis=0)
dd[icdcodes]=dd[icdcodes].apply(lambda x : x.dt.days)

for col in icdcodes[:len_icd+1]:
    X = dd[col].dropna().reset_index().to_numpy().astype(int)
    data_list.append(np.hstack((X,label_dict[col]*np.ones([X.shape[0],1],X.dtype))))

X = dd['sex'].reset_index().to_numpy().astype(int)
data_list.append(np.c_[X[:,0],np.zeros(X.shape[0]),X[:,1]].astype(int))

for j in range(17):
    dd_cancer = dd[['cancer_date_'+str(j),'cancer_type_'+str(j)]].dropna().reset_index()
    if not dd_cancer.empty:
        dd_cancer['cancer'] = dd_cancer['cancer_type_'+str(j)].str.slice(0,3)
        dd_cancer['cancer_label'] = dd_cancer["cancer"].map(label_dict)
        data_list.append(dd_cancer[['eid','cancer_date_'+str(j),'cancer_label']].dropna().astype(int).to_numpy())

#for j in range(213):
#    dd_hicd = dd[['hicd_date_'+str(j),'hicd_'+str(j)]].dropna().reset_index()
#    if not dd_hicd.empty:
#        dd_hicd['hicd'] = dd_hicd['hicd_'+str(j)].str.slice(0,3)
#        dd_hicd['hicd_label'] = dd_hicd["hicd"].map(label_dict)
#        data_list.append(dd_hicd[['f.eid','hicd_date_'+str(j),'hicd_label']].dropna().astype(int).to_numpy())
    
dd_bmi = dd[['assessment_date','BMI']].dropna().reset_index()
dd_bmi['bmi_status'] = np.where(dd_bmi['BMI']>28,5,np.where(dd_bmi.BMI>22,4,3))
data_list.append(dd_bmi[['eid','assessment_date','bmi_status']].astype(int).to_numpy())

dd_sm = dd[['assessment_date','smoking']].dropna().reset_index()
dd_sm = dd_sm[dd_sm['smoking']!=-3]
dd_sm['smoking_status'] = np.where(dd_sm['smoking']==1,8,np.where(dd_sm.smoking==2,7,6))
data_list.append(dd_sm[['eid','assessment_date','smoking_status']].astype(int).to_numpy())

dd_al = dd[['assessment_date','alcohol']].dropna().reset_index()
dd_al = dd_al[dd_al['alcohol']!=-3]
dd_al['alcohol_status'] = np.where(dd_al['alcohol']==1,11,np.where(dd_al.alcohol < 4,10,9))
data_list.append(dd_al[['eid','assessment_date','alcohol_status']].astype(int).to_numpy())

## reformat, split train and val and output to delphi format

In [13]:
data= np.vstack(data_list)
data = data[np.lexsort((data[:,1], data[:,2]==data[:,2].max(), data[:,0]))]
data = data[data[:,1]>=0]
data = pd.DataFrame(data).drop_duplicates([0,2]).values
data = data.astype(np.uint32)
data.tofile(output_prefix + '.bin')
ids = list(set(data[:,0]))
ids.sort()
train_val_split = data[:,0] <= ids[int(len(ids)*train_proportion)]
data[train_val_split].tofile(output_prefix + '_train.bin')
data[~train_val_split].tofile(output_prefix + '_val.bin')


In [33]:
!dx upload ukb_real_train.bin --path /users/la/data/delphi_v2_random_split_filt_all_train.bin
!dx upload ukb_real_val.bin --path /users/la/data/delphi_v2_random_split_filt_all_val.bin

[===========================================================>] Uploaded 79,622,568 of 79,622,568 bytes (100%) ukb_real_train.bin
ID                                file-J52125jJjp7bVGq1FbX6y8GQ
Class                             file
Project                           project-GvbfZj8Jjp7pv1F7KyVPGjp6
Folder                            /users/la/data
Name                              delphi_v2_random_split_filt_all_train.bin
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Thu Dec 18 14:35:03 2025
Created by                        larnoldt
 via the job                      job-J5209g0Jjp7p6048X1kxGjFK
Last modified                     Thu Dec 18 14:35:05 2025
Media type                        
archivalState                     "live"
cloudAccount                      "cloudaccount-

In [15]:
!dx download /users/la/notebooks/freeze3_eids.npy

[===========================================================>] Completed 78,618 of 78,618 bytes (100%) /opt/notebooks/Delphi/data/ukb_simulated_data/freeze3_eids.npyy


In [18]:
freeze3_eids = np.load("freeze3_eids.npy", allow_pickle=True).astype(np.uint32)
freeze3_eids = set(freeze3_eids.tolist())

In [22]:
all_eids = np.unique(data.T[0])

# Force freeze3 eids into train
train_eids = set(all_eids[np.isin(all_eids, list(freeze3_eids))])

# Remaining eids eligible for random split
remaining_eids = np.array(
    [eid for eid in all_eids if eid not in train_eids],
    dtype=np.uint32
)

In [25]:
rng = np.random.default_rng(seed=42)  # reproducible
rng.shuffle(remaining_eids)

n_train_remaining = int(0.8 * len(remaining_eids))

train_eids.update(remaining_eids[:n_train_remaining])
valid_eids = set(remaining_eids[n_train_remaining:])

In [26]:
assert train_eids.isdisjoint(valid_eids)
assert freeze3_eids.issubset(train_eids)

In [28]:
train_mask = np.isin(data.T[0], list(train_eids))
valid_mask = np.isin(data.T[0], list(valid_eids))

train_data = data[train_mask]
valid_data = data[valid_mask]

In [29]:
train_data.tofile('delphi_v2_sc_split_filt_all_train.bin')
valid_data.tofile('delphi_v2_sc_split_filt_all_val.bin')

In [32]:
!dx upload delphi_v2_sc_split_filt_all_train.bin --path /users/la/data/delphi_v2_sc_split_filt_all_train.bin
!dx upload delphi_v2_sc_split_filt_all_val.bin --path /users/la/data/delphi_v2_sc_split_filt_all_val.bin

[===========================================================>] Uploaded 79,853,448 of 79,853,448 bytes (100%) delphi_v2_sc_split_filt_all_train.bin
ID                                file-J521218Jjp7y065f0QGP6F5x
Class                             file
Project                           project-GvbfZj8Jjp7pv1F7KyVPGjp6
Folder                            /users/la/data
Name                              delphi_v2_sc_split_filt_all_train.bin
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Thu Dec 18 14:34:45 2025
Created by                        larnoldt
 via the job                      job-J5209g0Jjp7p6048X1kxGjFK
Last modified                     Thu Dec 18 14:34:47 2025
Media type                        
archivalState                     "live"
cloudAccount                     

In [34]:
train_sc_mask = np.isin(data.T[0], list(freeze3_eids))
valid_sc_mask = np.isin(data.T[0], [])

train_sc_data = data[train_sc_mask]
valid_sc_data = data[valid_sc_mask]

In [37]:
train_sc_data.tofile('delphi_v2_sc_split_filt_sc_train.bin')
valid_sc_data.tofile('delphi_v2_sc_split_filt_sc_val.bin')

In [38]:
!dx upload delphi_v2_sc_split_filt_sc_train.bin --path /users/la/data/delphi_v2_sc_split_filt_sc_train.bin
!dx upload delphi_v2_sc_split_filt_sc_val.bin --path /users/la/data/delphi_v2_sc_split_filt_sc_val.bin

[===========================================================>] Uploaded 833,376 of 833,376 bytes (100%) delphi_v2_sc_split_filt_sc_train.bin
ID                                file-J521308Jjp7vx0Z9fvY7j1K3
Class                             file
Project                           project-GvbfZj8Jjp7pv1F7KyVPGjp6
Folder                            /users/la/data
Name                              delphi_v2_sc_split_filt_sc_train.bin
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Thu Dec 18 14:36:49 2025
Created by                        larnoldt
 via the job                      job-J5209g0Jjp7p6048X1kxGjFK
Last modified                     Thu Dec 18 14:36:49 2025
Media type                        
archivalState                     "live"
cloudAccount                      "clouda

# Training

In [ ]:
%%bash

#git clone https://github.com/gerstung-lab/Delphi.git
git clone https://github.com/LArnoldt/Delphi.git
git checkout training_fixes
cd Delphi
pip install -r requirements.txt

mkdir ./data/ukb_real_data
cd ./data/ukb_real_data
dx download /users/la/data/delphi_v2_sc_split_filt_all_train.bin
dx download /users/la/data/delphi_v2_sc_split_filt_all_val.bin
cp delphi_v2_sc_split_filt_all_train.bin  train.bin
cp delphi_v2_sc_split_filt_all_val.bin val.bin
cd ../..
#Not required: -> Change: config/train_delphi.py  -> vocab_size = 1956 #1270
#Not required: -> Change: config/train_delphi.py  -> max_iters = 5000
#Not required: -> Change: config/train_delphi.py  -> lr_decay_iters = 5000 # make equal to max_iters usually

python train.py config/train_delphi.py --out_dir=Delphi-SingleCell --device=cuda 
python evaluate_auc.py --input_path=./data/ukb_real_data --model_ckpt_path=./Delphi-SingleCell/ckpt.pt --no_event_token_rate 5 --output_path Delphi-SingleCellAUC --dataset_subset_size 5000 

cd ./data/ukb_real_data
rm train.bin
rm val.bin
dx download /users/la/data/delphi_random_split_filt_sc_train.bin
dx download /users/la/data/delphi_random_split_filt_sc_val.bin
cp delphi_random_split_filt_sc_train.bin  train.bin
cp delphi_random_split_filt_sc_val.bin val.bin
python embedding.py config/train_delphi.py --out_dir=Delphi-SingleCell --device=cuda